In [1]:
import pandas as pd
from itertools import combinations,product

In [2]:
data_master = pd.read_excel("../0_dataset_master/Experiment_edited_V7_rr_with_correction.xlsx")
data_master.head()

,No,Start Date,End Date,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Experiment set,...,New_End_Date,Power VLF (ms2),Power LF (ms2),Power HF (ms2),Power Total (ms2),LF/HF,Fraction LF (nu),Fraction HF (nu),rr_mean,pmv
0,1,2023-07-10 10:18:12,2023-07-10 11:02:50,129.126.10.3,100,2677,True,2023-07-10 11:02:50,R_1QKIHfH2SzDYww5,set1,...,2023-07-10 11:05:00,2038.691441,1878.641717,874.282355,4791.615513,2.148781,68.241683,31.758317,884.113291,0.96
1,2,2023-07-10 09:07:40,2023-07-10 11:03:41,129.126.10.6,100,6961,True,2023-07-10 11:03:42,R_3oZ1X5rJydNMWKi,set1,...,2023-07-10 11:05:00,2038.691441,1878.641717,874.282355,4791.615513,2.148781,68.241683,31.758317,884.113291,1.01
2,3,2023-07-10 09:09:15,2023-07-10 11:03:52,129.126.10.8,100,6877,True,2023-07-10 11:03:53,R_3nvGWRYoFKSgygt,set1,...,2023-07-10 11:05:00,2038.691441,1878.641717,874.282355,4791.615513,2.148781,68.241683,31.758317,884.113291,-0.01
3,4,2023-07-10 10:18:07,2023-07-10 11:04:08,119.234.4.134,100,2761,True,2023-07-10 11:04:09,R_2U4WrS8qQBMNLbH,set1,...,2023-07-10 11:05:00,2038.691441,1878.641717,874.282355,4791.615513,2.148781,68.241683,31.758317,884.113291,0.15
4,5,2023-07-10 10:45:42,2023-07-10 11:05:05,119.56.107.228,100,1163,True,2023-07-10 11:05:06,R_2EhPoNBA5xgmC4D,set1,...,2023-07-10 11:05:00,2038.691441,1878.641717,874.282355,4791.615513,2.148781,68.241683,31.758317,884.113291,0.96


In [3]:
thermalUnique = list(data_master.Thermal.unique())
thermalCombin = combinations(thermalUnique,2)
thermalCombin = list(thermalCombin)
# print(thermalCombin)

TNLUnique = list(data_master['Traffic noise levels'].unique())
TNLCombin = combinations(TNLUnique,2)
TNLCombin = list(TNLCombin)
# print(TNLCombin)

TNLUnique.sort()
# print(TNLUnique)
firstProduct = product(TNLUnique[1:], thermalCombin)
firstProduct = list(firstProduct)

secondProduct = product(thermalUnique, TNLCombin)
secondProduct = list(secondProduct)

In [9]:
import statsmodels.formula.api as smf
from statsmodels.stats.mediation import Mediation

for item in firstProduct:
    # if item[1]==('NV fan 3', 'NV fan 0') or item[1]==('NV fan 0', 'NV fan 3'):
        sliced_data = data_master[(data_master['Traffic noise levels']==item[0]) & ((data_master['Thermal']==item[1][0]) | (data_master['Thermal']==item[1][1]))]
        mediate = smf.mixedlm("ThermalSatisfaction ~ SET",  data=sliced_data, groups=sliced_data['subid'])
        full_ = smf.mixedlm('DoubleTrouble_average ~ ThermalSatisfaction + SET',  data=sliced_data, groups=sliced_data['subid'])
        # complete_ = Mediation(full_, mediate, 'SET', mediator = 'ThermalSatisfaction').fit()
        # print(complete_.summary())
        # break

for item in secondProduct:
        try:
            sliced_data = data_master[(data_master['Thermal']==item[0]) & ((data_master['Traffic noise levels']==item[1][0]) | (data_master['Traffic noise levels']==item[1][1]))]
            mediate = smf.mixedlm("ThermalSatisfaction ~ SET",  data=sliced_data, groups=sliced_data['subid'])
            full_ = smf.mixedlm('DoubleTrouble_average ~ ThermalSatisfaction + SET',  data=sliced_data, groups=sliced_data['subid'])
            # complete_ = Mediation(full_, mediate, 'SET', mediator = 'ThermalSatisfaction').fit()
            # print(complete_.summary())
        except ValueError:
            print(item)
            print(sliced_data)
        

               Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: DoubleTrouble_average
No. Observations: 68      Method:             REML                 
No. Groups:       34      Scale:              15.0611              
Min. group size:  2       Log-Likelihood:     -242.5515            
Max. group size:  2       Converged:          Yes                  
Mean group size:  2.0                                              
-------------------------------------------------------------------
                       Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-------------------------------------------------------------------
Intercept             -49.855   48.825 -1.021 0.307 -145.551 45.841
ThermalSatisfaction     3.254    1.144  2.845 0.004    1.013  5.496
SET                     3.396    1.777  1.911 0.056   -0.086  6.878
Group Var             246.176   22.858                             

               Mixed Linear Model Regression Results
Model:   

c:\Users\ZENI\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
c:\Users\ZENI\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


               Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: DoubleTrouble_average
No. Observations: 68      Method:             REML                 
No. Groups:       34      Scale:              37.8448              
Min. group size:  2       Log-Likelihood:     -257.0778            
Max. group size:  2       Converged:          Yes                  
Mean group size:  2.0                                              
-------------------------------------------------------------------
                       Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-------------------------------------------------------------------
Intercept             -38.014   70.254 -0.541 0.588 -175.708 99.681
ThermalSatisfaction     0.613    1.475  0.415 0.678   -2.278  3.504
SET                     2.939    2.192  1.340 0.180   -1.358  7.236
Group Var             224.523   13.790                             

               Mixed Linear Model Regression Results
Model:   

c:\Users\ZENI\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
c:\Users\ZENI\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


               Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: DoubleTrouble_average
No. Observations: 68      Method:             REML                 
No. Groups:       34      Scale:              15.5819              
Min. group size:  2       Log-Likelihood:     -245.7800            
Max. group size:  2       Converged:          Yes                  
Mean group size:  2.0                                              
-------------------------------------------------------------------
                        Coef.  Std.Err.   z   P>|z|  [0.025  0.975]
-------------------------------------------------------------------
Intercept               39.798   58.607 0.679 0.497 -75.070 154.665
ThermalSatisfaction      1.193    0.906 1.317 0.188  -0.583   2.968
SET                      0.518    2.154 0.240 0.810  -3.704   4.741
Group Var              289.397   26.990                            

               Mixed Linear Model Regression Results
Model:   

c:\Users\ZENI\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
c:\Users\ZENI\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


               Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: DoubleTrouble_average
No. Observations: 68      Method:             REML                 
No. Groups:       34      Scale:              30.2829              
Min. group size:  2       Log-Likelihood:     -259.6495            
Max. group size:  2       Converged:          Yes                  
Mean group size:  2.0                                              
-------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025  0.975]
-------------------------------------------------------------------
Intercept              112.722   49.525  2.276 0.023 15.655 209.789
ThermalSatisfaction      0.958    0.959  0.999 0.318 -0.922   2.838
SET                     -2.483    2.052 -1.210 0.226 -6.505   1.540
Group Var              332.847   22.181                            

               Mixed Linear Model Regression Results
Model:   